In [44]:
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from nltk.lm.preprocessing import padded_everygram_pipeline

import pandas as pd
import re

In [45]:
dados_portugues = pd.read_csv('dados/stackoverflow_portugues_tratados.csv', sep=',')
dados_ingles = pd.read_csv('dados/stackoverflow_ingles_tratados.csv', sep=',')
dados_espanhol = pd.read_csv('dados/stackoverflow_espanhol_tratados.csv', sep=',')

### Fake Char

In [46]:
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import bigrams

test = bigrams('rodolfo')

print(list(bigrams(pad_both_ends('rodolfo', n=2))))

[('<s>', 'r'), ('r', 'o'), ('o', 'd'), ('d', 'o'), ('o', 'l'), ('l', 'f'), ('f', 'o'), ('o', '</s>')]


### Modelo de Linguagem

In [47]:
dados_portugues['idioma'] = 'port'
dados_ingles['idioma'] = 'ing'
dados_espanhol['idioma'] = 'esp'

In [48]:
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,questoes_tratadas,idioma
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,se eu fizer o hash de senhas antes de armazená...,port
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,qual é a diferença entre inner join e outer jo...,port
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,uma dúvida muito comum é por que devemos parar...,port
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,é comum encontrar uma mensagem de erro que diz...,port
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,me parecem termos muito próximos e eventualmen...,port


In [49]:
port_train, port_test = train_test_split(dados_portugues.questoes_tratadas, test_size=0.2, random_state=144)
ing_train, ing_test = train_test_split(dados_ingles.questoes_tratadas, test_size=0.2, random_state=144)
esp_train, esp_test = train_test_split(dados_espanhol.questoes_tratadas, test_size=0.2, random_state=144)

### Máxima verossimilhança

In [63]:
from nltk.lm import MLE

def treinar_modelo_mle(lista_textos):
    todas_questoes = ' '.join(lista_textos)
    todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
    bigramas, vocabulario = padded_everygram_pipeline(2, todas_palavras)
    modelo = MLE(2)
    modelo.fit(bigramas, vocabulario)
    return modelo

In [79]:
def calcular_perplexidade(modelo, textos):
    
    perplexidade_total = 0
    
    todas_palavras = WhitespaceTokenizer().tokenize(textos)
    palavras_fakechar = [list(pad_both_ends(palavra,n=2)) for palavra in todas_palavras ]
    palavras_bigramas = [list(bigrams(palavra)) for palavra in palavras_fakechar]
        
    for palavra in palavras_bigramas:
        perplexidade = modelo.perplexity(palavra)
        
        perplexidade_total += perplexidade
        
    return perplexidade_total
    

In [80]:
modelo_port_mle = treinar_modelo_mle(port_train)
modelo_ing_mle = treinar_modelo_mle(ing_train)
modelo_esp_mle = treinar_modelo_mle(esp_train)

In [81]:
calcular_perplexidade(modelo_esp_mle, port_test.iloc[3])

inf

### Modelo de Laplace

In [69]:
from nltk.lm import Laplace

def treinar_modelo_laplace(lista_textos):
    todas_questoes = ' '.join(lista_textos)
    todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
    bigramas, vocabulario = padded_everygram_pipeline(2, todas_palavras)
    modelo = Laplace(2)
    modelo.fit(bigramas, vocabulario)
    return modelo

In [82]:
modelo_port_laplace = treinar_modelo_laplace(port_train)
modelo_ing_laplace = treinar_modelo_laplace(ing_train)
modelo_esp_laplace = treinar_modelo_laplace(esp_train)

In [83]:
calcular_perplexidade(modelo_ing_laplace, port_test.iloc[3])

4490.550395727484

In [84]:
calcular_perplexidade(modelo_port_laplace, port_test.iloc[3])

1596.496133694623

In [85]:
calcular_perplexidade(modelo_esp_laplace, port_test.iloc[3])

2513.3123745498874

### Função Identificação do idioma

In [87]:
def atribui_idioma(lista_textos):
    
    idioma = []
    
    for texto in lista_textos:
        portugues = calcular_perplexidade(modelo_port_laplace, texto)
        ingles = calcular_perplexidade(modelo_ing_laplace, texto)
        espanhol = calcular_perplexidade(modelo_esp_laplace, texto)
        
        if ingles >= portugues <= espanhol:
            idioma.append('portugues')
        elif portugues > ingles < espanhol:
            idioma.append('ingles')
        else:
            idioma.append('espanhol')
        
    return idioma

In [97]:
atribui_idioma(['nos somos programadores','we are programmers','somos programadores'])

['portugues', 'ingles', 'portugues']

In [96]:
atribui_idioma(['Eu comi comida japonesa','I ate Japanese food','comí comida japonesa'])

['portugues', 'ingles', 'espanhol']

In [106]:
print('Portugues: ',atribui_idioma(port_test).count('portugues') / len(port_test))
print('Inglês: ',atribui_idioma(ing_test).count('ingles') / len(ing_test))
print('Espanhol: ', atribui_idioma(esp_test).count('espanhol') / len(esp_test))

Portugues:  0.97
Inglês:  1.0
Espanhol:  0.96
